# Behavioral Stability

In [1]:
import numpy as np
import scipy as sp
from scipy.optimize import minimize
import pandas as pd
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import ipynb.fs.defs.Functions_Equations_Equilibrium_Simulation as baseFuncs
from numpy import linalg as LA

### Jacobian of Behavioral System

In [2]:
def fu_prime_of_Ru(Ru, Ro, para):
    rd, ka, kc, kp, w = para['r_d'], para['k_A'], para['k_C'], para['k_P'], para['w_max'] 
    dtilde, gc, gp, a, b = para['dtilde'], para['gamma_c'], para['gamma_p'], para['a'], para['b']
    result = -rd + (ka*(b*ka*(Ro + Ru) - w)*np.exp(-((b*ka*(Ro + Ru))/w)))/ \
            (kc*w*np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**np.log(gc)*(dtilde + ((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**(-np.log(gp)) + np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**(-np.log(gc)))) - \
            (a*(b*ka*Ru - w)*np.exp(-((b*ka*(Ro + Ru))/w))*(w - ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))*np.log(gc))/ \
            (kc*w*np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**np.log(gc)*(dtilde + ((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**(-np.log(gp)) + np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**(-np.log(gc)))) - \
            ((w - ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))*(-((a*(b*ka*Ru - w)*np.exp(-((b*ka*(Ro + Ru))/w))*np.log(gc))/np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**np.log(gc)) + \
            ((b*ka*(Ro + Ru) - w)*np.log(gp))/((Ro + Ru)*((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**np.log(gp))))/ \
            (kc*w*np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**np.log(gc)*(dtilde + ((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**(-np.log(gp)) + np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**(-np.log(gc)))**2)
    
    return result

def fu_prime_of_Ro(Ru, Ro, para):
    rd, ka, kc, kp, w = para['r_d'], para['k_A'], para['k_C'], para['k_P'], para['w_max'] 
    dtilde, gc, gp, a, b = para['dtilde'], para['gamma_c'], para['gamma_p'], para['a'], para['b']

    result = (ka*(b*ka*(Ro + Ru) - w)*np.exp(-((b*ka*(Ro + Ru))/w))*(dtilde + ((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**(-np.log(gp)) + np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**(-np.log(gc))) + \
              a*b*ka*Ru*np.exp(-((b*ka*(Ro + Ru))/w))*(-w + ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))*\
              (dtilde + ((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**(-np.log(gp)) + np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**(-np.log(gc)))*np.log(gc) - \
              ((-w + ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))*((a*b*ka*Ru*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w))*np.log(gc))/np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**np.log(gc) + \
            ((-(b*ka*(Ro + Ru)) + w)*np.log(gp))/((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**np.log(gp)))/(Ro + Ru))/ \
            (kc*w*np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**np.log(gc)*(dtilde + ((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**(-np.log(gp)) + np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**(-np.log(gc)))**2)
    return result

def fo_prime_of_Ru(Ru, Ro, para):
    rd, ka, kc, kp, w = para['r_d'], para['k_A'], para['k_C'], para['k_P'], para['w_max'] 
    dtilde, gc, gp, a, b = para['dtilde'], para['gamma_c'], para['gamma_p'], para['a'], para['b']

    result =  rd - (ka**2*Ro*(b*ka*(Ro + Ru) - w)*np.exp((-2*b*ka*(Ro + Ru))/w)*((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**(-1 - np.log(gp)))/(kp*w**2*(dtilde + ((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**(-np.log(gp)) + np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**(-np.log(gc)))) + \
        (Ro*(w - ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w))))/(kp*(Ro + Ru)**2*((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**np.log(gp)*(dtilde + ((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**(-np.log(gp)) + np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**(-np.log(gc)))) + \
        (Ro*(-(b*ka*(Ro + Ru)) + w)*(w - ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))*np.log(gp))/ \
        (kp*(Ro + Ru)**2*w*((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**np.log(gp)*(dtilde + ((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**(-np.log(gp)) + np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**(-np.log(gc)))) - \
        (Ro*np.exp((-2*b*ka*(Ro + Ru))/w)*(ka*(Ro + Ru) - w*np.exp((b*ka*(Ro + Ru))/w))*np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**np.log(gc)\
         *(-(a*(Ro + Ru)*(b*ka*Ru - w)*((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**np.log(gp)*np.log(gc)) + (b*ka*(Ro + Ru) - w)*np.exp((b*ka*(Ro + Ru))/w)*np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**np.log(gc)*np.log(gp)))/ \
        (kp*(Ro + Ru)**2*w*(((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**np.log(gp) + np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**np.log(gc) + dtilde*((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**np.log(gp)*np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**np.log(gc))**2)
    return result

def fo_prime_of_Ro(Ru, Ro, para):
    rd, ka, kc, kp, w = para['r_d'], para['k_A'], para['k_C'], para['k_P'], para['w_max'] 
    dtilde, gc, gp, a, b = para['dtilde'], para['gamma_c'], para['gamma_p'], para['a'], para['b']

    result = (-(ka*Ro*(Ro + Ru)*(b*ka*(Ro + Ru) - w)*np.exp(-((b*ka*(Ro + Ru))/w))*(dtilde + ((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**(-np.log(gp)) + \
            np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**(-np.log(gc)))) - Ro*w*(-w + ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))*\
              (dtilde + ((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**(-np.log(gp)) + np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**(-np.log(gc))) + \
              (Ro + Ru)*w*(-w + ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))*(dtilde + ((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**(-np.log(gp)) + np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**(-np.log(gc))) + \
              Ro*(b*ka*(Ro + Ru) - w)*(-w + ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))*(dtilde + ((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**(-np.log(gp)) + \
            np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**(-np.log(gc)))*np.log(gp) + Ro*(-w + ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))*\
              ((a*b*ka*Ru*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w))*np.log(gc))/np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**np.log(gc) + \
               ((-(b*ka*(Ro + Ru)) + w)*np.log(gp))/((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**np.log(gp)))/\
            (kp*(Ro + Ru)**2*w*((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**np.log(gp)*\
             (dtilde + ((ka*(Ro + Ru)*np.exp(-((b*ka*(Ro + Ru))/w)))/w)**(-np.log(gp)) + np.exp(-(a*Ru*np.exp(-((b*ka*(Ro + Ru))/w))))**(-np.log(gc)))**2)
    return result

def jacobian_eignvalues(Rs, para, only_stab = False):
    """
    get the eigenvalues of the Jacobian of the dynamical system 
    and stability at the fixed point(Ru, Ro)
    """
    Ru, Ro = Rs
    # construct Jacobian matrix 
    J11 = fu_prime_of_Ru(Ru, Ro, para)
    J12 = fu_prime_of_Ro(Ru, Ro, para)
    J21 = fo_prime_of_Ru(Ru, Ro, para)
    J22 = fo_prime_of_Ro(Ru, Ro, para)
    J = np.array([[J11, J12], [J21, J22]])

    # compute eigenvalues
    lambdas, eigenvectors = LA.eig(J)
    
    # determine convergence
    if len(lambdas) != 2:
         print("warning: did not return 2 eigen values")
         stab = np.nan
    elif (lambdas[0] <= 0) and (lambdas[1]<= 0): # 2 negative eigenvalues -> fixed point stable. 
        stab = 1
    elif (lambdas[0] >0) or (lambdas[1] > 0):  # 1 positive eigenvalue -> fixed point unstable. 
        stab = 0
    else: 
        stab = np.nan

    if only_stab:
        return stab
    else:
        return stab, lambdas